In [3]:
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch
from pathlib import Path
import re
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.to(device)

# pipe.unet.load_attn_procs("patrickvonplaten/lora_dreambooth_dog_example")
pipe.unet.load_attn_procs("normal_to_malignant_image2image")
seed = 42
generator = torch.Generator(device=device).manual_seed(seed)
text_prompt = "ultrasound image of breast with cancer malignant"
image0 = pipe(text_prompt, num_inference_steps=25,generator=generator).images[0]


def slugify(text):
   text = re.sub(r"[^\w\s]", "", text)
   text = re.sub(r"\s+", "-", text)
   return text


DIR_NAME="./saved_images/imgtoimg/images_generated_text_to_image/"
dirpath_text_to_image = Path(DIR_NAME)
# create parent dir if doesn't exist
dirpath_text_to_image.mkdir(parents=True, exist_ok=True)


image_name = f'{slugify(text_prompt)}.png'
image_path = dirpath_text_to_image / image_name
image0.save(image_path)

# for idx, (image,prompt) in enumerate(zip(image0.images, prompts*num_images_per_prompt)):
#     image_name = f'{slugify(prompt)}-{idx}.png'
#     image_path = dirpath / image_name
#     image.save(image_path)


100%|██████████| 25/25 [00:01<00:00, 14.85it/s]


In [4]:

from diffusers  import StableDiffusionImg2ImgPipeline, EulerDiscreteScheduler
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch


device = "cuda" if torch.cuda.is_available() else "cpu"
# Step 2: Create the pipeline with your UNet model
model_id =  "stabilityai/stable-diffusion-2"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16)



# pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
# pipe.to(device)

pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

pipe.unet.load_attn_procs("normal_to_malignant_image2image")



Loading pipeline components...: 100%|██████████| 6/6 [00:01<00:00,  5.16it/s]


StableDiffusionImg2ImgPipeline {
  "_class_name": "StableDiffusionImg2ImgPipeline",
  "_diffusers_version": "0.20.0.dev0",
  "_name_or_path": "stabilityai/stable-diffusion-2",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "requires_safety_checker": false,
  "safety_checker": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}



In [5]:
# pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
# pipe.to("cuda")
from PIL import Image
from pathlib import Path
import torch
import re

steps = 20
scale = 9
num_images_per_prompt = 1
# seed = torch.randint(0, 1000000, (1,)).item()
seed = 9
generator = torch.Generator(device=device).manual_seed(seed)

# images = ["/home/brianformento/Dataset_BUSI_with_GT/normal/normal (1).png",
# "/home/brianformento/Dataset_BUSI_with_GT/benign/benign (1).png" ,
#  "/home/brianformento/Dataset_BUSI_with_GT/malignant/malignant (1).png"]

images = ["./ultrasound/normal/normal (5).png",
"./ultrasound/normal/normal (6).png" ,
 "./ultrasound/normal/normal (7).png"]

init_images = [Image.open(image).convert("RGB").resize((768,768)) for image in images] 



prompts = ["ultra sound image of breast with malignant cancer",
           "ultra sound image of breast with malignant cancer", 
           "ultra sound image of breast with malignant cancer"]


DIR_NAME="./saved_images/imgtoimg/images_preprocessed/"
dirpath_preprocessed = Path(DIR_NAME)
# create parent dir if doesn't exist
dirpath_preprocessed.mkdir(parents=True, exist_ok=True)

for idx, (image,prompt) in enumerate(zip(init_images, prompts )):
    image_name = f'{slugify(prompt)}-{idx}.png'
    image_path = dirpath_preprocessed / image_name
    image.save(image_path)

negative_prompts = ["ultrasound scanning device",
                    "ultrasound scanning device",
                    "ultrasound scanning device"]

output =  pipe(prompts, negative_prompt=negative_prompts, image=init_images, num_inference_steps=steps,
guidance_scale=scale, num_images_per_prompt=num_images_per_prompt, generator=generator)



DIR_NAME="./saved_images/imgtoimg/images_generated_image_to_image/"
dirpath = Path(DIR_NAME)
# create parent dir if doesn't exist
dirpath.mkdir(parents=True, exist_ok=True)

def slugify(text):
   text = re.sub(r"[^\w\s]", "", text)
   text = re.sub(r"\s+", "-", text)
   return text

for idx, (image,prompt) in enumerate(zip(output.images, prompts*num_images_per_prompt)):
    image_name = f'{slugify(prompt)}-{idx}.png'
    image_path = dirpath / image_name
    image.save(image_path)


100%|██████████| 16/16 [00:02<00:00,  6.19it/s]
